In [3]:
# Calculate the Gini index for groups

def gini_index(groups, classes):

  n_instances = float(sum([len(group) for group in groups]))

  gini = 0.0

  for group in groups:

      size = float(len(group))

      if size == 0:

           continue

      score = 0.0

      labels = [row[-1] for row in group]

  for class_val in classes:

       p = labels.count(class_val) / size

       score += p * p

  gini += (1.0 - score) * (size / n_instances)

  return gini


In [7]:
def test_split(index, value, dataset):

  left, right = [], []

  for row in dataset:

      if row[index] < value:

           left.append(row)

      else:

           right.append(row)

  return left, right

In [14]:
def get_best_split(dataset):

  class_values = list(set(row[-1] for row in dataset))

  best_index, best_value, best_score, best_groups = None, None, float('inf'), None

  for index in range(len(dataset[0]) - 1):

      for row in dataset:

           groups = test_split(index, row[index], dataset)

           gini = gini_index(groups, class_values)

           if gini < best_score:

                best_index, best_value, best_score, best_groups = index, row[index], gini, groups

  return {'index': best_index, 'value': best_value, 'groups': best_groups}


In [13]:
def to_terminal(group):

  outcomes = [row[-1] for row in group]

  return Counter(outcomes).most_common(1)[0][0]

In [20]:
def split(node, max_depth, min_size, depth):

      left, right = node['groups']

      del(node['groups'])

      # Check for no split

      if not left or not right:

          node['left'] = node['right'] = to_terminal(left + right)

      return

      # Check max depth

      if depth >= max_depth:

          node['left'], node['right'] = to_terminal(left), to_terminal(right)

      return

      # Process left child

      if len(left) <= min_size:

          node['left'] = to_terminal(left)

      else:

          node['left'] = get_best_split(left)

          split(node['left'], max_depth, min_size, depth + 1)

      # Process right child

      if len(right) <= min_size:

          node['right'] = to_terminal(right)

      else:

          node['right'] = get_best_split(right)

      split(node['right'], max_depth, min_size, depth + 1)

In [21]:
def build_tree(train, max_depth, min_size):

      root = get_best_split(train)

      split(root, max_depth, min_size, 1)

      return root

In [41]:
def predict(node, row):

  if row[node['index']] < node['value']:
    if isinstance(node['left'], dict):
        return predict(node['left'], row)
    else:
        return node['left']
  else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']